011 データを読み込んでみる

In [2]:
import pandas as pd
uriage_data = pd.read_csv('./input/02/uriage.csv')
uriage_data.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美
1,2019-07-13 13:05:29,商 品 S,NaN,浅田賢二
2,2019-05-11 19:42:07,商 品 a,NaN,南部慶二
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒
4,2019-04-22 03:09:35,商品a,NaN,平田鉄二


In [5]:
uriage_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   purchase_date  2999 non-null   object 
 1   item_name      2999 non-null   object 
 2   item_price     2612 non-null   float64
 3   customer_name  2999 non-null   object 
dtypes: float64(1), object(3)
memory usage: 93.8+ KB


`item_price`に欠損データ`NaN`のデータあることや`商`と`品`の間にスペースが入っているなど表記の揺れが大きいのが見て取れる  
欠損データがある列は`float64`と判断される

In [4]:
print(pd.__version__)
kokyaku_data = pd.read_excel('./input/02/kokyaku_daicho.xlsx')
kokyaku_data.head()

1.3.0


,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018/01/04
1,岡田 敏也,おかだ としや,E市,okada_toshiya@example.com,42782
2,芳賀 希,はが のぞみ,A市,haga_nozomi@example.com,2018/01/07
3,荻野 愛,おぎの あい,F市,ogino_ai@example.com,42872
4,栗田 憲一,くりた けんいち,E市,kurita_kenichi@example.com,43127


In [7]:
kokyaku_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   顧客名      200 non-null    object
 1   かな       200 non-null    object
 2   地域       200 non-null    object
 3   メールアドレス  200 non-null    object
 4   登録日      200 non-null    object
dtypes: object(5)
memory usage: 7.9+ KB


欠損データはないが、スペースが入っていたり、登録日のフォーマットがまちまちだったりする

012 データの揺れを確認


In [8]:
print(uriage_data['item_name'])

0         商品A
1       商 品 S
2       商 品 a
3         商品Z
4         商品a
        ...  
2994      商品Y
2995      商品M
2996      商品Q
2997      商品H
2998      商品D
Name: item_name, Length: 2999, dtype: object


In [9]:
print(uriage_data['item_price'])

0        100.0
1          NaN
2          NaN
3       2600.0
4          NaN
         ...  
2994    2500.0
2995    1300.0
2996       NaN
2997       NaN
2998     400.0
Name: item_price, Length: 2999, dtype: float64


013 データが欠損のまま集計

In [17]:
uriage_data['purchase_date']
uriage_data['purchase_date'] = pd.to_datetime(uriage_data['purchase_date'])
uriage_data['purchase_month'] = uriage_data['purchase_date'].dt.strftime("%Y%m")
res = uriage_data.pivot_table(index='purchase_month', columns='item_name', aggfunc='size', fill_value=0)
res

item_name,商品W,商 品 n,商品E,商品M,商品P,商品S,商品W,商品X,商 品O,商 品Q,...,商品k,商品l,商品o,商品p,商品r,商品s,商品t,商品v,商品x,商品y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,0,1,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,1,1,0,0
201903,0,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
201905,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
201906,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
201907,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,2,0,0,0,0,0


In [20]:
res = uriage_data.pivot_table(index='purchase_month', columns='item_name', values='item_price', aggfunc='sum', fill_value=0)
res

item_name,商品W,商 品 n,商品E,商品M,商品P,商品S,商品W,商品X,商 品O,商 品Q,...,商品k,商品l,商品o,商品p,商品r,商品s,商品t,商品v,商品x,商品y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,0,1400,0,0,0,0,0,0,0,0,...,1100,1200,1500,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,0,2400,0,0,...,0,0,0,0,0,1900,2000,2200,0,0
201903,0,0,500,1300,1600,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,2300,0,0,0,0,0,0,0,0,1700,...,0,0,0,0,0,1900,0,0,0,0
201905,0,0,0,0,0,1900,0,0,0,0,...,0,1200,0,0,0,0,0,0,0,2500
201906,0,0,0,0,0,0,2300,0,0,0,...,0,0,0,1600,0,0,0,0,2400,0
201907,0,0,0,0,0,0,0,0,0,0,...,0,0,1500,0,1800,0,0,0,0,0


014 商品名の揺れを補正する

In [24]:
print(pd.unique(uriage_data['item_name']))
print(len(pd.unique(uriage_data.item_name)))

['商品A' '商 品 S' '商 品 a' '商品Z' '商品a' '商品S' '商品 a' '商品V' '商品O' '商 品U' '商品L'
 '商  品V' '商 品O' '商品C' '商品I' '商品r' '商品X' '商品 g' '商品R' '商品P' '商品Q' '商品y'
 '商品 A' '商品N' '商品W' '商 品E' '商品K' '商品B' '商品F' '商 品s' '  商品W' ' 商 品 n'
 '商 品F' '商品D' '商品M' '商品Y' '商品U' '商品H' '商品T' '商品J' '商  品O' '商品E' '商  品Q'
 ' 商品S' ' 商品M' '商  品T' '商品G' '商 品G' ' 商品P' ' 商品E' '商 品N' '商 品Y' '商品 J'
 '商品 V' '商品 K' '商 品V' '商 品D' '商 品A' '商品 F' '商品 H' '商 品K' '商 品T' '商品 X'
 '商品 Q' '商 品X' '商 品H' '商 品C' '商品 B' '商品 O' '商品 T' '商品v' '商品p' '商品i' '商品 w'
 '商 品 s' '商 品 q' '商品s' '商品l' '商品t' '商品k' '商品g' '商品o' '商品 R' '商品 S' '商 品M'
 '商品j' '商品d' '商品 I' '商品 E' '商品 o' '商品c' '商品 v' '商品e' '商品x' '商 品I' ' 商品W'
 ' 商品X' '商品 M' '商 品P']
99


戦略として、全角と半角スペースを除く、英字を半角大文字化すれば統一できそう

In [25]:
uriage_data['item_name'] = uriage_data['item_name'].str.upper()
uriage_data['item_name'] = uriage_data['item_name'].str.replace('　', '')
uriage_data['item_name'] = uriage_data['item_name'].str.replace(' ', '')
print(pd.unique(uriage_data['item_name']))
print(len(pd.unique(uriage_data.item_name)))

['商品A' '商品S' '商品Z' '商品V' '商品O' '商品U' '商品L' '商品C' '商品I' '商品R' '商品X' '商品G'
 '商品P' '商品Q' '商品Y' '商品N' '商品W' '商品E' '商品K' '商品B' '商品F' '商品D' '商品M' '商品H'
 '商品T' '商品J']
26


商品名の揺れが補正されている

015 金額の欠損値を補完しよう

In [27]:
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price         True
customer_name     False
purchase_month    False
dtype: bool

`item_price`に欠損があることがわかる。一覧で見ると結構抜けている

In [29]:
uriage_data['item_price']

0        100.0
1          NaN
2          NaN
3       2600.0
4          NaN
         ...  
2994    2500.0
2995    1300.0
2996       NaN
2997       NaN
2998     400.0
Name: item_price, Length: 2999, dtype: float64

In [45]:
flg_is_null = uriage_data['item_price'].isnull()
for trg in list(uriage_data.loc[flg_is_null, 'item_name'].unique()):
    price = uriage_data.loc[(~flg_is_null) & (uriage_data['item_name'] == trg), 'item_price'].max()
    uriage_data.loc[(flg_is_null) & (uriage_data['item_name'] == trg), 'item_price'] = price
    
uriage_data.isnull().any(axis=0)



purchase_date     False
item_name         False
item_price        False
customer_name     False
purchase_month    False
dtype: bool

最大値、最小値を確認してデータの補完状況を確認する

In [48]:
for trg in list(uriage_data['item_name'].sort_values().unique()):
    print(trg + 'の最大値:' + str(uriage_data.loc[uriage_data['item_name']==trg]['item_price'].max()) + ' 最小値:' + str(uriage_data.loc[uriage_data['item_name']==trg]['item_price'].min(skipna=False)))


商品Aの最大値:100.0 最小値:100.0
商品Bの最大値:200.0 最小値:200.0
商品Cの最大値:300.0 最小値:300.0
商品Dの最大値:400.0 最小値:400.0
商品Eの最大値:500.0 最小値:500.0
商品Fの最大値:600.0 最小値:600.0
商品Gの最大値:700.0 最小値:700.0
商品Hの最大値:800.0 最小値:800.0
商品Iの最大値:900.0 最小値:900.0
商品Jの最大値:1000.0 最小値:1000.0
商品Kの最大値:1100.0 最小値:1100.0
商品Lの最大値:1200.0 最小値:1200.0
商品Mの最大値:1300.0 最小値:1300.0
商品Nの最大値:1400.0 最小値:1400.0
商品Oの最大値:1500.0 最小値:1500.0
商品Pの最大値:1600.0 最小値:1600.0
商品Qの最大値:1700.0 最小値:1700.0
商品Rの最大値:1800.0 最小値:1800.0
商品Sの最大値:1900.0 最小値:1900.0
商品Tの最大値:2000.0 最小値:2000.0
商品Uの最大値:2100.0 最小値:2100.0
商品Vの最大値:2200.0 最小値:2200.0
商品Wの最大値:2300.0 最小値:2300.0
商品Xの最大値:2400.0 最小値:2400.0
商品Yの最大値:2500.0 最小値:2500.0
商品Zの最大値:2600.0 最小値:2600.0


016: 顧客名の揺れを補正する

In [50]:
kokyaku_data['顧客名'].head()

0     須賀ひとみ
1    岡田　 敏也
2      芳賀 希
3     荻野  愛
4     栗田 憲一
Name: 顧客名, dtype: object

In [53]:
uriage_data['customer_name'].head()

0    深井菜々美
1     浅田賢二
2     南部慶二
3     麻生莉緒
4     平田鉄二
Name: customer_name, dtype: object

In [54]:
kokyaku_data['顧客名'] = kokyaku_data['顧客名'].str.replace('　', '')
kokyaku_data['顧客名'] = kokyaku_data['顧客名'].str.replace(' ', '')
kokyaku_data['顧客名'].head()

0    須賀ひとみ
1     岡田敏也
2      芳賀希
3      荻野愛
4     栗田憲一
Name: 顧客名, dtype: object

In [62]:
print(kokyaku_data['顧客名'].unique())
print(uriage_data['customer_name'].unique())

['須賀ひとみ' '岡田敏也' '芳賀希' '荻野愛' '栗田憲一' '梅沢麻緒' '相原ひとり' '新村丈史' '石川まさみ' '小栗正義'
 '大倉晃司' '那須蒼甫' '清水裕次郎' '榊原しぼり' '高沢美咲' '川島友以乃' '唐沢景子' '稲田将也' '秋葉あき' '西脇礼子'
 '内村まさみ' '進藤瞬' '小口豊' '笹原しぼり' '岩佐孝太郎' '河村由樹' '菅原誠治' '榎本薫' '高梨結衣' '鶴岡薫'
 '藤木一恵' '森岡季衣' '田畑正敏' '宇野秀樹' '高原充則' '梅村秀樹' '浅田賢二' '石渡小雁' '吉岡サダヲ' '井口寛治'
 '奥光洋' '佐藤慶二' '東光博' '高田さんま' '合田光' '深田信輔' '島英嗣' '梅津淳子' '日比野徹' '熊倉綾' '松田浩正'
 '楠高史' '岡慶太' '市田寿明' '山西花' '青山鉄洋' '松川綾女' '大滝麗奈' '赤木愛梨' '楠哲平' '堀内聡' '八木雅彦'
 '林勇' '日野夏希' '麻生莉緒' '根岸仁晶' '赤木だん吉' '亀井一徳' '横田遥' '野本仁晶' '西原未華子' '五十嵐春樹'
 '堀江佑' '久保田倫子' '金森なつみ' '城戸芳正' '望月真悠子' '井本マサカズ' '米沢仁晶' '本橋直人' '古川信吾' '岩沢那奈'
 '西島知世' '戸塚美幸' '深井照生' '新谷智花' '有馬徹平' '本多フミヤ' '浜田未華子' '牧田玲那' '島孝太郎' '石田花'
 '星野美嘉' '黒谷長利' '深沢ひろ子' '矢部夏空' '立石茜' '手塚雅之' '松谷愛子' '中田美智子' '片瀬長利' '石崎幸子'
 '明石家明' '小川美菜' '田辺光洋' '神原美嘉' '岩城徹平' '相川良介' '植村遥' '井川真悠子' '松元翔太' 'さだ千佳子'
 '島本研二' '影山輝信' '浅見広司' '石野仁' '小平陽子' '福本美幸' '中仁晶' '田辺きみまろ' '熊倉明日' '鈴木一哉'
 '杉田将也' '徳重優' '松岡ノブヒコ' '藤広之' '谷本愛梨' '松沢育二' '清水佑' '土屋朝陽' '鳥居広司' '福島友也'
 '堀サンタマリア' '井川里穂' '田崎菜々美' '平田鉄二' '井上桃子' '尾形小雁' '沖遥' '矢部美幸' '島袋

017: 日付の揺れを補正する

顧客台帳はExcelで作成されており、人が入力しているため日付入力にも揺れがある

In [63]:
print(uriage_data.columns)
print(kokyaku_data.columns)


Index(['purchase_date', 'item_name', 'item_price', 'customer_name',
       'purchase_month'],
      dtype='object')
Index(['顧客名', 'かな', '地域', 'メールアドレス', '登録日'], dtype='object')


In [65]:
print(kokyaku_data['登録日'])

0      2018/01/04
1           42782
2      2018/01/07
3           42872
4           43127
          ...    
195    2017/06/20
196    2018/06/20
197    2017/04/29
198    2019/04/19
199    2019/04/23
Name: 登録日, Length: 200, dtype: object


In [73]:
flg_is_serial= kokyaku_data['登録日'].astype('str').str.isdigit()
flg_is_serial= kokyaku_data['登録日'].astype('str').str.isdigit()
print(flg_is_serial)

0      False
1       True
2      False
3       True
4       True
       ...  
195    False
196    False
197    False
198    False
199    False
Name: 登録日, Length: 200, dtype: bool


In [74]:
from_serial = pd.to_timedelta(kokyaku_data.loc[flg_is_serial, '登録日'].astype('float64'), unit='D') + pd.to_datetime('1900/1/1')
from_serial

1     2017-02-18
3     2017-05-19
4     2018-01-29
21    2017-07-06
27    2017-06-17
47    2017-01-08
49    2017-07-15
53    2017-04-10
76    2018-03-31
80    2018-01-12
99    2017-06-01
114   2018-06-05
118   2018-01-31
122   2018-04-18
139   2017-05-27
143   2017-03-26
155   2017-01-21
172   2018-03-24
179   2017-01-10
183   2017-07-26
186   2018-07-15
192   2018-06-10
Name: 登録日, dtype: datetime64[ns]

In [75]:
from_string = pd.to_datetime(kokyaku_data.loc[~flg_is_serial, '登録日'])
from_string

0     2018-01-04
2     2018-01-07
5     2017-06-20
6     2018-06-11
7     2017-05-19
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 登録日, Length: 178, dtype: datetime64[ns]

In [76]:
kokyaku_data['登録日'] = pd.concat([from_serial, from_string])
kokyaku_data

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04
1,岡田敏也,おかだ としや,E市,okada_toshiya@example.com,2017-02-18
2,芳賀希,はが のぞみ,A市,haga_nozomi@example.com,2018-01-07
3,荻野愛,おぎの あい,F市,ogino_ai@example.com,2017-05-19
4,栗田憲一,くりた けんいち,E市,kurita_kenichi@example.com,2018-01-29
...,...,...,...,...,...
195,川上りえ,かわかみ りえ,G市,kawakami_rie@example.com,2017-06-20
196,小松季衣,こまつ としえ,E市,komatsu_toshie@example.com,2018-06-20
197,白鳥りえ,しらとり りえ,F市,shiratori_rie@example.com,2017-04-29
198,大西隆之介,おおにし りゅうのすけ,H市,oonishi_ryuunosuke@example.com,2019-04-19


登録月の列を作成して集計を行う

In [79]:
kokyaku_data['登録年月'] = kokyaku_data['登録日'].dt.strftime("%Y%m")
result = kokyaku_data.groupby('登録年月').count()['顧客名']
result

登録年月
201701    15
201702    11
201703    14
201704    15
201705    13
201706    14
201707    17
201801    13
201802    15
201803    17
201804     5
201805    19
201806    13
201807    17
201904     2
Name: 顧客名, dtype: int64

In [80]:
flg_is_serial = kokyaku_data['登録日'].astype('str').str.isdigit()
flg_is_serial.sum()

0

018 顧客名をキーにデータを結合する

In [82]:
join_data = pd.merge(uriage_data, kokyaku_data, left_on='customer_name', right_on='顧客名', how='left')
join_data.drop('customer_name', axis=1)
join_data

,purchase_date,item_name,item_price,customer_name,purchase_month,顧客名,かな,地域,メールアドレス,登録日,登録年月
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,201906,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26,201701
1,2019-07-13 13:05:29,商品S,1900.0,浅田賢二,201907,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07,201804
2,2019-05-11 19:42:07,商品A,100.0,南部慶二,201905,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19,201806
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒,201902,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22,201807
4,2019-04-22 03:09:35,商品A,100.0,平田鉄二,201904,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07,201706
...,...,...,...,...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:39,商品Y,2500.0,福島友也,201902,福島友也,ふくしま ともや,B市,fukushima_tomoya@example.com,2017-07-01,201707
2995,2019-06-22 04:03:43,商品M,1300.0,大倉晃司,201906,大倉晃司,おおくら こうじ,E市,ookura_kouji@example.com,2018-03-31,201803
2996,2019-03-29 11:14:05,商品Q,1700.0,尾形小雁,201903,尾形小雁,おがた こがん,B市,ogata_kogan@example.com,2017-03-15,201703
2997,2019-07-14 12:56:49,商品H,800.0,芦田博之,201907,芦田博之,あしだ ひろゆき,E市,ashida_hiroyuki@example.com,2018-07-15,201807


019 クレンジングしたデータをダンプしておく

In [83]:
dump_data = join_data[['purchase_date', 'purchase_month', 'item_name', 'item_price', '顧客名', 'かな', '地域', 'メールアドレス', '登録日']]
dump_data

,purchase_date,purchase_month,item_name,item_price,顧客名,かな,地域,メールアドレス,登録日
0,2019-06-13 18:02:34,201906,商品A,100.0,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26
1,2019-07-13 13:05:29,201907,商品S,1900.0,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07
2,2019-05-11 19:42:07,201905,商品A,100.0,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19
3,2019-02-12 23:40:45,201902,商品Z,2600.0,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22
4,2019-04-22 03:09:35,201904,商品A,100.0,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07
...,...,...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:39,201902,商品Y,2500.0,福島友也,ふくしま ともや,B市,fukushima_tomoya@example.com,2017-07-01
2995,2019-06-22 04:03:43,201906,商品M,1300.0,大倉晃司,おおくら こうじ,E市,ookura_kouji@example.com,2018-03-31
2996,2019-03-29 11:14:05,201903,商品Q,1700.0,尾形小雁,おがた こがん,B市,ogata_kogan@example.com,2017-03-15
2997,2019-07-14 12:56:49,201907,商品H,800.0,芦田博之,あしだ ひろゆき,E市,ashida_hiroyuki@example.com,2018-07-15


In [84]:
dump_data.to_csv('./input/02/dump_data.csv', index=False)

020 データを集計する

In [85]:
import_data = pd.read_csv('./input/02/dump_data.csv')
import_data

,purchase_date,purchase_month,item_name,item_price,顧客名,かな,地域,メールアドレス,登録日
0,2019-06-13 18:02:34,201906,商品A,100.0,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26 00:00:00
1,2019-07-13 13:05:29,201907,商品S,1900.0,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07 00:00:00
2,2019-05-11 19:42:07,201905,商品A,100.0,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19 00:00:00
3,2019-02-12 23:40:45,201902,商品Z,2600.0,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22 00:00:00
4,2019-04-22 03:09:35,201904,商品A,100.0,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07 00:00:00
...,...,...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:39,201902,商品Y,2500.0,福島友也,ふくしま ともや,B市,fukushima_tomoya@example.com,2017-07-01 00:00:00
2995,2019-06-22 04:03:43,201906,商品M,1300.0,大倉晃司,おおくら こうじ,E市,ookura_kouji@example.com,2018-03-31 00:00:00
2996,2019-03-29 11:14:05,201903,商品Q,1700.0,尾形小雁,おがた こがん,B市,ogata_kogan@example.com,2017-03-15 00:00:00
2997,2019-07-14 12:56:49,201907,商品H,800.0,芦田博之,あしだ ひろゆき,E市,ashida_hiroyuki@example.com,2018-07-15 00:00:00


In [86]:
by_item = import_data.pivot_table(index='purchase_month', columns='item_name', values='item_price', aggfunc='size', fill_value=0)
by_item

item_name,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,...,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,18,13,19,17,18,15,11,16,18,17,...,17,21,20,17,7,22,13,14,10,0
201902,19,14,26,21,16,14,14,17,12,14,...,22,22,22,23,19,22,24,16,11,1
201903,17,21,20,17,9,27,14,18,12,16,...,23,16,20,12,23,18,16,21,16,0
201904,17,19,24,20,18,17,14,11,18,13,...,20,20,16,16,11,15,14,16,20,0
201905,24,14,16,14,19,18,23,15,16,11,...,13,22,18,16,16,9,21,16,20,0
201906,24,12,11,19,13,18,15,13,19,22,...,15,16,21,12,18,20,17,15,13,0
201907,20,20,17,17,12,17,19,19,19,23,...,15,19,23,21,13,28,16,18,12,0


In [87]:
by_price = import_data.pivot_table(index='purchase_month', columns='item_name', values='item_price', aggfunc='sum', fill_value=0)
by_price

item_name,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,...,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,1800,2600,5700,6800,9000,9000,7700,12800,16200,17000,...,28900,37800,38000,34000,14700,48400,29900,33600,25000,0
201902,1900,2800,7800,8400,8000,8400,9800,13600,10800,14000,...,37400,39600,41800,46000,39900,48400,55200,38400,27500,2600
201903,1700,4200,6000,6800,4500,16200,9800,14400,10800,16000,...,39100,28800,38000,24000,48300,39600,36800,50400,40000,0
201904,1700,3800,7200,8000,9000,10200,9800,8800,16200,13000,...,34000,36000,30400,32000,23100,33000,32200,38400,50000,0
201905,2400,2800,4800,5600,9500,10800,16100,12000,14400,11000,...,22100,39600,34200,32000,33600,19800,48300,38400,50000,0
201906,2400,2400,3300,7600,6500,10800,10500,10400,17100,22000,...,25500,28800,39900,24000,37800,44000,39100,36000,32500,0
201907,2000,4000,5100,6800,6000,10200,13300,15200,17100,23000,...,25500,34200,43700,42000,27300,61600,36800,43200,30000,0


In [91]:
by_customer = import_data.pivot_table(index='purchase_month', columns='顧客名', values='item_price', aggfunc='size', fill_value=0)
by_customer

顧客名,さだ千佳子,中仁晶,中田美智子,丸山光臣,久保田倫子,亀井一徳,五十嵐春樹,井上桃子,井口寛治,井川真悠子,...,香椎優一,高原充則,高梨結衣,高沢美咲,高田さんま,鳥居広司,鶴岡薫,麻生莉緒,黄川田博之,黒谷長利
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,3,1,4,2,2,0,5,3,3,1,...,0,1,1,1,5,2,0,2,2,5
201902,9,1,2,2,1,4,2,1,0,4,...,4,0,3,2,0,1,2,4,0,1
201903,1,2,1,6,1,4,3,3,2,2,...,3,1,6,2,4,2,4,2,2,1
201904,0,3,1,2,0,2,2,0,3,2,...,2,4,2,3,4,3,2,1,2,0
201905,3,2,5,2,4,1,2,1,3,3,...,1,1,1,0,2,2,3,4,4,1
201906,1,3,0,4,1,1,1,2,2,3,...,7,3,0,2,1,0,2,1,2,4
201907,3,0,3,2,5,3,5,2,5,5,...,2,4,4,2,0,2,4,3,4,1


In [90]:
by_region = import_data.pivot_table(index='purchase_month', columns='地域', values='item_price', aggfunc='size', fill_value=0)
by_region

地域,A市,B市,C市,D市,E市,F市,G市,H市
purchase_month,,,,,,,,
201901,59,55,72,34,49,57,49,42
201902,71,46,65,48,61,52,43,63
201903,64,52,57,43,52,59,51,59
201904,64,48,54,45,48,58,40,52
201905,57,52,68,48,59,65,35,43
201906,53,47,61,30,51,51,58,58
201907,76,53,61,42,54,64,47,54


期間内に購入のない顧客

In [96]:
away_data = pd.merge(uriage_data, kokyaku_data, left_on='customer_name', right_on='顧客名', how='right')
print(away_data)
away_data[away_data.purchase_date.isnull()][['顧客名', 'メールアドレス', '登録日']]

           purchase_date item_name  item_price customer_name purchase_month  \
0    2019-02-24 01:07:56       商品C       300.0         須賀ひとみ         201902   
1    2019-05-08 15:42:01       商品P      1600.0         須賀ひとみ         201905   
2    2019-07-03 07:49:05       商品M      1300.0         須賀ひとみ         201907   
3    2019-01-02 13:52:15       商品L      1200.0         須賀ひとみ         201901   
4    2019-06-29 04:58:36       商品R      1800.0         須賀ひとみ         201906   
...                  ...       ...         ...           ...            ...   
2995 2019-02-17 06:40:12       商品S      1900.0         大西隆之介         201902   
2996 2019-04-23 02:16:45       商品G       700.0         大西隆之介         201904   
2997 2019-06-15 00:31:12       商品J      1000.0         大西隆之介         201906   
2998 2019-07-17 23:00:47       商品K      1100.0         大西隆之介         201907   
2999                 NaT       NaN         NaN           NaN            NaN   

        顧客名           かな  地域                       

,顧客名,メールアドレス,登録日
2999,福井美希,fukui_miki1@example.com,2019-04-23
